In [1]:
from astropy import units as u
from astropy.time import Time, TimeDelta
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")
from poliastro.bodies import Sun, Earth, Moon
from poliastro.ephem import Ephem
from poliastro.util import time_range
import matplotlib.pyplot as plt
import datetime
import random
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
import math
from poliastro.plotting import OrbitPlotter3D
from astropy.utils.data import conf
import numpy as np

In [2]:
#Finds date where body is a distance d away
def find_date(d):
    low = 0 
    high = mission_length
    mid = 0
    
    while low <= high:
        mid = (high + low) / 2
        sc_pos = np.array([ss_trans.propagate(date_launch + TimeDelta(mid * u.day)).sample()[0].xyz])
        body_pos = np.array([ss_target.propagate(date_launch + TimeDelta(mid * u.day)).sample()[0].xyz])
        pos = sc_pos - body_pos
        dist = int(np.linalg.norm(pos))
        
        #print(date_launch + TimeDelta(mid * u.day))
        if dist > d:
            low = mid
        elif dist < d:
            high = mid
        else:
            #print(dist)
            print(date_launch + TimeDelta(mid * u.day))
            return (date_launch + TimeDelta(mid * u.day))

In [3]:
#Gives dx where 100 * dx = end - start
def find_dx(start, end):
    return float(str((end - start)/ 100))

In [4]:
#Filters and saves 100 data points between min_distance and max_distance
def filtered_positions(min_distance, max_distance):
    start_date = find_date(max_distance)
    end_date = find_date(min_distance)
    dx = find_dx(start_date, end_date)
    print(start_date)
    print(end_date)
    #print(dx)
    i = 0
    f_sc_launch_pos = np.array([ss_trans.propagate(start_date + TimeDelta(i * dx * u.day)).sample()[0].xyz])
    f_body_launch_pos = np.array([ss_target.propagate(start_date + TimeDelta(i * dx * u.day)).sample()[0].xyz])
    f_cur_pos = f_sc_launch_pos - f_body_launch_pos
    filtered_pos = f_cur_pos
    #print(np.linalg.norm(f_cur_pos))
    i += 1
    
    while(i < 100):
        f_cur_pos_sc = np.array([ss_trans.propagate(start_date + TimeDelta(i * dx * u.day)).sample()[0].xyz])
        f_cur_pos_body = np.array([ss_target.propagate(start_date + TimeDelta(i * dx * u.day)).sample()[0].xyz])
        filtered_pos = np.append(filtered_pos, f_cur_pos_sc - f_cur_pos_body, axis = 0)
        #print(np.linalg.norm(f_cur_pos_sc - f_cur_pos_body))
        i += 1
    #print("filtered complete")
    print(filtered_pos)
    return filtered_pos
    
    #print(filtered_pos)

In [5]:
def R_2vect(vector_orig, vector_fin):
    R = np.zeros((3,3))
    # Convert the vectors to unit vectors.
    vector_orig = vector_orig / np.linalg.norm(vector_orig)
    vector_fin = vector_fin / np.linalg.norm(vector_fin)
    # The rotation axis (normalised).
    axis = np.cross(vector_orig, vector_fin)
    axis_len = np.linalg.norm(axis)
    if axis_len != 0.0:
        axis = axis / axis_len
    # Alias the axis coordinates.
    x = axis[0]
    y = axis[1]
    z = axis[2]
    # The rotation angle.
    angle = math.acos(np.dot(vector_orig, vector_fin))
    # Trig functions (only need to do this maths once!).
    ca = math.cos(angle)
    sa = math.sin(angle)
    # Calculate the rotation matrix elements.
    R[0,0] = 1.0 + (1.0 - ca)*(x**2 - 1.0)
    R[0,1] = -z*sa + (1.0 - ca)*x*y
    R[0,2] = y*sa + (1.0 - ca)*x*z
    R[1,0] = z*sa+(1.0 - ca)*x*y
    R[1,1] = 1.0 + (1.0 - ca)*(y**2 - 1.0)
    R[1,2] = -x*sa+(1.0 - ca)*y*z
    R[2,0] = -y*sa+(1.0 - ca)*x*z
    R[2,1] = x*sa+(1.0 - ca)*y*z
    R[2,2] = 1.0 + (1.0 - ca)*(z**2 - 1.0)
    return R
def rotationMatrixToEulerAngles(R) :
    #assert(isRotationMatrix(R))
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
    singular = sy < 1e-6
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
    return np.array([x, y, z])

def calculateEuler(path):
    orbits_attitudes = None
    vector_orig = [0,0,1]
    vec_orig_flip = -1*np.array(vector_orig)
    pos = np.load(path + '.npy')
    for k in range(len(pos)):
    # pos_ is the position vector from the target body to the camera (pos_ = camera_position - target_body_position)
        for i in range(100):
            pos_ = -1*pos[k][i]
            R = R_2vect(vec_orig_flip, pos_) 
            angle0_blend = rotationMatrixToEulerAngles(R)
            if i == 0:
                euler_pos_xyz = np.array([angle0_blend])
            else:
                euler_pos_xyz = np.append(euler_pos_xyz, [angle0_blend], axis = 0)
    #print(euler_pos_xyz)
        if orbits_attitudes is None:
            orbits_attitudes = np.array([euler_pos_xyz])
        else:
            orbits_attitudes = np.append(orbits_attitudes, [euler_pos_xyz], axis = 0)
        np.save('orbits_attitudes', orbits_attitudes)

#calculateEuler('pos_xyz')


In [6]:
random.seed("SSDS")
dates = []
asteroid_choice = []
orbits_positions = None

In [7]:
for i in range(0,175):
    chosen_body = random.randint(0,3)
    asteroid_choice.append(chosen_body)
    if chosen_body == 0:
        chosen_body = "Bennu"
    elif chosen_body == 1:
        chosen_body = "Itokawa"
    elif chosen_body == 2:
        chosen_body = "Mithra"
    elif chosen_body == 3:
        chosen_body = "Toutatis"
    
    #Launch Date
    launch_year = random.randint(2000,2019)
    launch_month = random.randint(1,12)
    launch_day = random.randint(1,28)

    #Arrival Date
    arrival_year = random.randint(launch_year+1, launch_year + 5)
    arrival_month = random.randint(launch_month,12)
    arrival_day = random.randint(launch_day, 28)
    
    #Converting to dates
    l_dt = datetime.date(launch_year, launch_month, launch_day)
    a_dt = datetime.date(arrival_year, arrival_month, arrival_day)
    mission_length = (a_dt - l_dt).days
    
    #print(a_dt > l_dt)
    #print(chosen_body)

    launch_date_string = str(launch_year) + "-" + str(launch_month) + "-" + str(launch_day)
    arrival_date_string = str(arrival_year) + "-" + str(arrival_month) + "-" + str(arrival_day)
    
    dates.append(launch_date_string + " " + arrival_date_string)
    #print(launch_date_string + " " + arrival_date_string)
    #print(datetime.datetime.now())
    date_launch = Time(launch_date_string, scale="tdb")
    date_arrival = Time(arrival_date_string, scale="tdb")
    points = 100

    #Filters
    min_distance = 50000
    max_distance = 500000
    
    #Ephemerides for Body and Earth
    body = Ephem.from_horizons(chosen_body, time_range(date_launch, end=date_arrival, periods=points), attractor=Sun)
    earth = Ephem.from_body(Earth, time_range(date_launch, end=date_arrival, periods=points))

    #Orbits for Body and Earth
    ss_body = Orbit.from_ephem(Sun, body, date_arrival)
    ss_earth = Orbit.from_ephem(Sun, earth, date_launch)
    
    

    #Lambert maneuver between from Earth to Body
    man_lambert = Maneuver.lambert(ss_earth, ss_body, short=False)

    #Orbits of spacecraft and Body
    ss_trans, ss_target = ss_earth.apply_maneuver(man_lambert, intermediate=True)
    
    sc_launch_pos = np.array([ss_trans.propagate(date_launch).sample()[0].xyz])
    body_launch_pos = np.array([ss_target.propagate(date_launch).sample()[0].xyz])

    sc_arrival_pos = np.array([ss_trans.propagate(date_arrival).sample()[0].xyz])
    body_arrival_pos = np.array([ss_target.propagate(date_arrival).sample()[0].xyz])

    days = 1
    cur_pos = sc_launch_pos - body_launch_pos
    pos = cur_pos

    #print("orbits calculated")
    '''
    while(days <= mission_length):
        cur_pos_sc = np.array([ss_trans.propagate(date_launch + TimeDelta(days * u.day)).sample()[0].xyz])
        cur_pos_body = np.array([ss_target.propagate(date_launch + TimeDelta(days * u.day)).sample()[0].xyz])
        pos = np.append(pos, cur_pos_sc - cur_pos_body, axis = 0)
        days += 1
    '''
    #print("filtered")
    filtered_pos = filtered_positions(min_distance, max_distance)
    #print(np.shape(filtered_pos))
    #print(np.shape(orbits_positions))
    if orbits_positions is None:
        orbits_positions = np.array([filtered_pos])
    else:
        orbits_positions = np.append(orbits_positions, [filtered_pos], axis = 0)
    #np.save('pos_xyz', pos)
    #np.save('filtered_pos_xyz', filtered_pos)
    print(i)
#print(orbits_positions)
np.save('orbits_positions', orbits_positions)
np.save('asteroid_choice', asteroid_choice)
calculateEuler('orbits_positions')

C:\Users\srika\anaconda3\lib\site-packages\astroquery\jplhorizons\core.py:1048: UserWarning: The uri used in this query is very long and might have been truncated. The results of the query might be compromised. If you queried a list of epochs, consider querying a range.
  warnings.warn(('The uri used in this query is very long '


2003-11-18 20:07:59.878
2003-11-18 23:36:47.962
2003-11-18 20:07:59.878
2003-11-18 23:36:47.962
[[ 379766.41697446 -298123.1780549  -130002.62655348]
 [ 376348.50894064 -295440.06051517 -128832.59738162]
 [ 372930.60137376 -292756.94336832 -127662.56842531]
 [ 369512.69426985 -290073.82660879 -126492.53968184]
 [ 366094.7876247  -287390.71023329 -125322.51114931]
 [ 362676.88143405 -284707.59423687 -124152.48282529]
 [ 359258.97569372 -282024.47861709 -122982.45470823]
 [ 355841.07039918 -279341.36337247 -121812.42679697]
 [ 352423.16554661 -276658.24849713 -120642.39908864]
 [ 349005.26113158 -273975.1339879  -119472.37158149]
 [ 345587.35715032 -271292.01983993 -118302.34427308]
 [ 342169.45359817 -268608.9060511  -117132.31716194]
 [ 338751.55047084 -265925.79261877 -115962.29024653]
 [ 335333.64776401 -263242.67953828 -114792.26352444]
 [ 331915.74547364 -260559.56680639 -113622.23699385]
 [ 328497.84359561 -257876.45441779 -112452.21065213]
 [ 325079.94212554 -255193.34237053 -111

C:\Users\srika\anaconda3\lib\site-packages\astroquery\jplhorizons\core.py:1048: UserWarning: The uri used in this query is very long and might have been truncated. The results of the query might be compromised. If you queried a list of epochs, consider querying a range.
  warnings.warn(('The uri used in this query is very long '


2011-10-27 18:30:42.407
2011-10-27 23:27:04.212
2011-10-27 18:30:42.407
2011-10-27 23:27:04.212
[[226328.70729291 445580.57644999 -15288.37656485]
 [224291.71514978 441570.32973379 -15150.81340878]
 [222254.72400902 437560.08377162 -15013.24938624]
 [220217.73386721 433549.83855741 -14875.68450662]
 [218180.74471024 429539.59408391 -14738.11877634]
 [216143.75652926 425529.35034415 -14600.55220328]
 [214106.76931685 421519.10733174 -14462.98479568]
 [212069.78306325 417508.86503983 -14325.41656109]
 [210032.79775858 413498.62346153 -14187.84750719]
 [207995.81339473 409488.38259006 -14050.277642  ]
 [205958.82996012 405478.14241847 -13912.70697261]
 [203921.84745166 401467.90294075 -13775.13550849]
 [201884.86585417 397457.66414934 -13637.5632558 ]
 [199847.88516054 393447.4260377  -13499.99022281]
 [197810.90535928 389437.18859877 -13362.41641675]
 [195773.92644824 385426.95182659 -13224.84184733]
 [193736.94840924 381416.7157134  -13087.26651978]
 [191699.97124156 377406.48025322 -12

2018-08-20 07:58:38.920
2018-08-22 17:35:51.676
2018-08-20 07:58:38.920
2018-08-22 17:35:51.676
[[358040.49244179 244149.39839661 249397.23318592]
 [354818.14019437 241951.94886923 247152.63354525]
 [351595.78771293 239754.50224984 244908.03463373]
 [348373.4349983  237557.05851167 242663.4364447 ]
 [345151.08205338 235359.61762792 240418.83897138]
 [341928.72888023 233162.17957348 238174.24220717]
 [338706.37547988 230964.74432117 235929.64614582]
 [335484.02185571 228767.31184494 233685.05078048]
 [332261.66800788 226569.88211709 231440.45610398]
 [329039.31393944 224372.45511329 229195.8621107 ]
 [325816.95965257 222175.03080648 226951.26879367]
 [322594.60514894 219977.60916895 224706.67614567]
 [319372.25042932 217780.19017512 222462.08416045]
 [316149.89549792 215582.7737999  220217.49283174]
 [312927.54035529 213385.36001468 217972.90215257]
 [309705.18500383 211187.94879496 215728.31211653]
 [306482.82944547 208990.54011303 213483.72271666]
 [303260.47368212 206793.13394362 211

...

2020-03-24 11:27:45.674
2020-03-24 22:44:46.525
2020-03-24 11:27:45.674
2020-03-24 22:44:46.525
[[ 48447.58800102 456896.33784705 197228.00345373]
 [ 48011.55983194 452784.26534235 195452.9491201 ]
 [ 47575.53167224 448672.19310069 193677.89489859]
 [ 47139.50352102 444560.12111998 191902.84078813]
 [ 46703.47537927 440448.04939801 190127.7867879 ]
 [ 46267.44724618 436335.97793233 188352.73289686]
 [ 45831.41912166 432223.90672088 186577.67911407]
 [ 45395.39100578 428111.83576053 184802.62543821]
 [ 44959.36289831 423999.76504964 183027.5718686 ]
 [ 44523.33479937 419887.69458503 181252.51840386]
 [ 44087.30670843 415775.62436467 179477.46504313]
 [ 43651.2786261  411663.5543865  177702.41178554]
 [ 43215.250552   407551.48464823 175927.35863021]
 [ 42779.22248594 403439.41514617 174152.30557537]
 [ 42343.19442736 399327.34587938 172377.25262076]
 [ 41907.16637725 395215.2768448  170602.19976506]
 [ 41471.13833489 391103.20804065 168827.14700747]
 [ 41035.11030009 386991.13946372 167

2008-12-26 11:41:03.217
2008-12-26 22:46:06.263
2008-12-26 11:41:03.217
2008-12-26 22:46:06.263
[[-401354.32959564 -281595.0046556   -98079.00302482]
 [-397742.14711381 -279060.64984524  -97196.29166624]
 [-394129.96459536 -276526.29509282  -96313.58033586]
 [-390517.78204831 -273991.9404512   -95430.86905658]
 [-386905.59947231 -271457.58591515  -94548.15782601]
 [-383293.41686259 -268923.23145092  -93665.44662955]
 [-379681.23422599 -266388.87709433  -92782.73548278]
 [-376069.05155735 -263854.52281988  -91900.02437434]
 [-372456.86886007 -261320.16863966  -91017.31331   ]
 [-368844.68613586 -258785.81456232  -90134.6022926 ]
 [-365232.50337781 -256251.46054465  -89251.89130452]
 [-361620.32059526 -253717.10664093  -88369.18036836]
 [-358008.13778308 -251182.75281358  -87486.46946806]
 [-354395.95494202 -248648.39906782  -86603.75860602]
 [-350783.77207434 -246114.0454129   -85721.04778594]
 [-347171.58917946 -243579.69184941  -84838.33700824]
 [-343559.40625563 -241045.33835679  -83

2012-11-25 20:53:49.253
2012-11-25 23:41:22.918
2012-11-25 20:53:49.253
2012-11-25 23:41:22.918
[[ 38750.63232003 458087.61124703 196607.92336512]
 [ 38401.86532407 453964.81627551 194838.44937836]
 [ 38053.09864044 449842.02156389 193068.9755004 ]
 [ 37704.33226769 445719.22711064 191299.50173065]
 [ 37355.56620419 441596.43291447 189530.02806857]
 [ 37006.80044682 437473.6389731  187760.55451318]
 [ 36658.03498942 433350.84528171 185991.08106245]
 [ 36309.26983173 429228.05183958 184221.60771614]
 [ 35960.50497271 425105.2586457  182452.13447374]
 [ 35611.74040784 420982.46569613 180682.66133367]
 [ 35262.97613361 416859.67298867 178913.18829497]
 [ 34914.21214451 412736.88051955 177143.71535613]
 [ 34565.44844346 408614.0882892  175374.24251724]
 [ 34216.68502527 404491.29629444 173604.76977699]
 [ 33867.92188689 400368.50453222 171835.29713415]
 [ 33519.15902221 396245.71299885 170065.82458709]
 [ 33170.3964345  392122.92169532 168296.35213634]
 [ 32821.63411798 388000.13061744 166

2019-12-13 23:48:52.478
2019-12-26 14:22:03.775
2019-12-13 23:48:52.478
2019-12-26 14:22:03.775
[[  35981.24701756 -412747.74883926 -279902.61808318]
 [  35659.32500067 -409029.43175709 -277382.48856732]
 [  35337.35134315 -405311.21449816 -274862.38874504]
 [  35015.32652444 -401593.096228   -272342.318371  ]
 [  34693.25102061 -397875.07611215 -269822.27719989]
 [  34371.12530953 -394157.15331626 -267302.26498654]
 [  34048.9498691  -390439.32700044 -264782.28148374]
 [  33726.72517565 -386721.59632784 -262262.32644549]
 [  33404.45170689 -383003.96045661 -259742.39962399]
 [  33082.12994039 -379286.41854721 -257222.50077221]
 [  32759.76035225 -375568.96975613 -254702.62964183]
 [  32437.34341916 -371851.61323977 -252182.78598434]
 [  32114.87961775 -368134.34815323 -249662.9695513 ]
 [  31792.36942479 -364417.17365056 -247143.18009326]
 [  31469.8133162  -360700.08888328 -244623.41736022]
 [  31147.21176767 -356983.09300286 -242103.68110216]
 [  30824.56525597 -353266.1851598  -239

2007-10-21 18:20:26.331
2007-10-21 23:26:02.604
2007-10-21 18:20:26.331
2007-10-21 23:26:02.604
[[490312.12371996 -72542.35275906 -65818.1989803 ]
 [485899.30473998 -71889.4706477  -65225.83509475]
 [481486.48621926 -71236.58859044 -64633.47123767]
 [477073.66815412 -70583.7065867  -64041.10740883]
 [472660.85054088 -69930.82463729 -63448.74360855]
 [468248.03337362 -69277.94273867 -62856.37983503]
 [463835.21664977 -68625.06089288 -62264.01608935]
 [459422.40036494 -67972.17909878 -61671.65237088]
 [455009.58451477 -67319.29735698 -61079.28867993]
 [450596.76909506 -66666.41566477 -60486.92501513]
 [446183.95410129 -66013.53402265 -59894.56137671]
 [441771.13952938 -65360.65242938 -59302.19776405]
 [437358.32537615 -64707.77088733 -58709.83417825]
 [432945.5116362  -64054.88939276 -58117.47061757]
 [428532.69830602 -63402.00794595 -57525.10708204]
 [424119.88538107 -62749.12654717 -56932.74357178]
 [419707.07285777 -62096.24519562 -56340.38008641]
 [415294.2607317  -61443.36389122 -55